In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import warnings

from sklearn.model_selection import train_test_split
from tqdm import notebook

warnings.filterwarnings(action='ignore')

In [ ]:
root = "/workspace/EmotionShortForm/data_audio" # 오디오 파일을 저장할 폴더
os.chdir(root)

In [ ]:
# 어노테이션 csv 파일들이 들어있는 폴더 경로
annotations_folder = "annotation"

# 모든 csv 파일을 리스트에 저장
csv_files = [os.path.join(annotations_folder, f) for f in os.listdir(annotations_folder) if f.endswith(".csv")]

# 모든 데이터를 저장할 데이터프레임
all_data = pd.DataFrame()

# 각 csv 파일에서 데이터를 추출하여 데이터프레임에 추가
for csv_file in csv_files:
    data = pd.read_csv(csv_file, header=1)
    data["time"] = data["end"]-data["start"]
    data.insert(4, "time", data.pop("time"))
    data = data.iloc[:, 3:8]
    all_data = all_data.append(data, ignore_index=True)

# . -> SegmentId 로 첫 번째 컬럼명 변경
all_data = all_data.rename(columns={all_data.columns[0]: 'SegmentId'})

# all_data 를 csv 파일로 저장
all_data.to_csv("/workspace/EmotionShortForm/data_audio/all_data.csv")

# train set과 test set으로 데이터를 나눔
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42)

# train set을 CSV 파일로 저장
train_data.to_csv("/workspace/EmotionShortForm/data_audio/train.csv")

# test set을 CSV 파일로 저장
test_data.to_csv("/workspace/EmotionShortForm/data_audio/test.csv")

In [ ]:
import shutil

# wav 파일들이 들어있는 폴더 경로입니다.
wav_folder_path = 'wav_only'

# train 폴더에 복사할 wav 파일들이 들어있는 폴더 경로입니다.
train_folder_path = 'train'

# test 폴더에 복사할 wav 파일들이 들어있는 폴더 경로입니다.
test_folder_path = 'test'

# train 폴더가 없으면 생성합니다.
if not os.path.exists(train_folder_path):
    os.makedirs(train_folder_path)

# test 폴더가 없으면 생성합니다.
if not os.path.exists(test_folder_path):
    os.makedirs(test_folder_path)

# train.csv에 있는 SegmentId들을 리스트로 저장합니다.
train_segment_ids = train_data['SegmentId'].tolist()

# test.csv에 있는 SegmentId들을 리스트로 저장합니다.
test_segment_ids = test_data['SegmentId'].tolist()

# wav 폴더에 있는 모든 wav 파일들에 대해서
for wav_file_name in notebook.tqdm(os.listdir(wav_folder_path),desc = 'iterate range 100'):
    
    # 파일 이름에서 확장자를 제외한 부분만 가져옵니다.
    segment_id = os.path.splitext(wav_file_name)[0]
    src_path = os.path.join(wav_folder_path, wav_file_name)

    # 해당 파일이 train.csv에 있는 경우에만 train 폴더로 복사합니다.
    if segment_id in train_segment_ids:
        dst_path = os.path.join(train_folder_path, wav_file_name)
    
    # 해당 파일이 test.csv에 있는 경우에만 test 폴더로 복사합니다.
    if segment_id in test_segment_ids:
        dst_path = os.path.join(test_folder_path, wav_file_name)
    
    shutil.copy(src_path, dst_path)


iterate range 100:   0%|          | 0/13467 [00:00<?, ?it/s]

In [ ]:
a = len(all_data)

trainfp = len(os.listdir(train_folder_path))
testfp = len(os.listdir(test_folder_path))

# train test split 확인
if a == (trainfp+testfp):
    print('파일 수가 동일합니다.')
else:
    print('파일 수가 동일하지않습니다.')

파일 수가 동일합니다.


In [ ]:
trainfp = len(os.listdir('/workspace/EmotionShortForm/data_audio/train'))
trainfp

10769

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import tensorflow as tf
from tqdm import tqdm
from glob import glob
from google.colab import drive

import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')

In [ ]:
root = '/workspace/EmotionShortForm'

# 캡스톤 폴더를 root로 변경
os.chdir(root)

In [ ]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [ ]:
audio_data = 'audio_data'

audio_data_train = os.path.join(audio_data,'train')
audio_data_test = os.path.join(audio_data,'test')

train = pd.read_csv(os.path.join(audio_data,'train.csv'))
test = pd.read_csv(os.path.join(audio_data,'test.csv'))

In [ ]:
train.head()

,Unnamed: 0,SegmentId,time,Emotion,Valence,Arousal
0,11164,Sess26_script04_User052F_010,7.785990,neutral,3.0,3.5
1,13031,Sess24_script03_User047F_009,12.188990,happy,3.4,3.6
2,1147,Sess09_script01_User018M_030,4.147000,neutral,2.6,3.1
3,6435,Sess14_script02_User028M_002,1.556999,neutral,3.6,3.1
4,7053,Sess32_script01_User064F_049,5.443000,neutral,3.0,3.5


In [ ]:
test.head()

,Unnamed: 0,SegmentId,time,Emotion,Valence,Arousal
0,63,Sess13_script01_User026F_029,7.254990,happy,3.7,3.5
1,5163,Sess03_script03_User005M_021,2.212000,surprise,2.9,3.4
2,7844,Sess23_script02_User046M_022,9.281020,neutral,2.3,3.0
3,304,Sess13_script05_User026F_017,6.614991,happy,4.4,4.2
4,5216,Sess03_script04_User005M_008,3.404998,angry,1.6,3.2


# 오디오 데이터 불러오기

In [ ]:
sr = 20000

In [ ]:
for i in os.listdir(audio_data_train)[:3]:
  print(f'{i}')
  data, sample_rate = librosa.load(os.path.join(audio_data_train,i), sr=sr)
  print('sample_rate:', sample_rate, ', audio shape:', data.shape)
  print('length:', data.shape[0]/float(sample_rate), 'secs')
  print('\n')

Sess38_script04_User076F_016.wav
sample_rate: 20000 , audio shape: (166600,)
length: 8.33 secs


Sess38_script05_User075M_009.wav
sample_rate: 20000 , audio shape: (576520,)
length: 28.826 secs


Sess38_script05_User075M_006.wav
sample_rate: 20000 , audio shape: (98100,)
length: 4.905 secs




In [ ]:
from tqdm.notebook import tqdm

train_file_names = train["SegmentId"].to_numpy()
test_file_names = test["SegmentId"].to_numpy()
target = train["Emotion"].to_numpy()

def load_audio(file_names, target, path):   
  audios = []
  for audio in tqdm(file_names):
    # librosa를 이용하여 데이터 로드
    an_audio, _ = librosa.load(os.path.join(path,f'{audio}.wav'), sr=sr)
    audio_array = np.array(an_audio)
    audios.append(audio_array)
  audios = np.array(audios)

  targets = target.copy()

  return audios, targets


In [ ]:
audio_train, target_train = load_audio(train_file_names, target, path=audio_data_train)
audio_test, _ = load_audio(test_file_names, np.array([None]), path=audio_data_test)

  0%|          | 0/10769 [00:00<?, ?it/s]

  0%|          | 0/2693 [00:00<?, ?it/s]

In [ ]:
# load된 numpy 파일들 저장
np.save(audio_data, audio_train)
np.save(audio_data, target_train)
np.save(audio_data, audio_test)